На публичной части тестового датасета лучшие результаты достигнуты:
- ResNet152, набор аугментаций включая блюр, Adam, LR 3e4, batch 64: 95.629
- ResNet50, без аугментаций, Adam, LR 3e4, BS64: 95.695

# Подготовка, импорты библиотек

In [30]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


import torchvision
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

import PIL
from PIL import Image
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [31]:
# Hyperparameters
NUM_CLASSES = 67
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
NUM_EPOCHS = 20
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

In [32]:
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

# Сбор датасетов для обучения, валидации и теста

In [33]:
#указываем путь к данным
DATA_PATH = "./data/"
TRAIN_ANN_PATH = DATA_PATH + 'train.csv'

#читаем аннотацию
train_df = pd.read_csv(TRAIN_ANN_PATH)
print(train_df.head(2))

     filename  class_number
0  000000.png            18
1  000001.png            18


In [34]:
class RoadSignDataset(data.Dataset):
  """Road Signs dataset class.

    Arguments:
        root (str): path to images
        imlist - pandas DataFrame with columns file_name, class
        transform - torchvision transform applied to every image
    """
  def __init__(self, root, flist, transform=None):
        self.root   = root
        self.imlist = flist 
        self.transform = transform

  def __getitem__(self, index):
        #берем строку из пришедшего df по index
        impath, target = self.imlist.loc[index]

        #собираем полное имя картинки
        full_imname = os.path.join(self.root, impath)

        if not os.path.exists(full_imname):
            print('No file ', full_imname)
            pass

        img = Image.open(full_imname).convert('RGB')

        #применяем к изображению выбранное преобразование (аугментацию)
        img = self.transform(img)

        #на выход отдаём img, target - нужны для обучения и валидации
        return img, target, impath

  #метод возвращает длину датасета - просто как длину подаваемого dataframe
  def __len__(self):
        return len(self.imlist)

In [35]:
class RoadSignTestDataset(data.Dataset):
  """Road Signs Test dataset class.

    Arguments:
        root (str): path to images
        imlist - list of file_name
        transform - torchvision transform applied to every image
    """
  def __init__(self, root, flist=None, transform=None):
        self.root   = root
        
        if flist is not None:
            self.imlist = flist
        else:
            self.imlist = []
            for filename in os.listdir(self.root):
                if filename[filename.rfind(".") + 1:] in ['jpg', 'jpeg', 'png']:
                    self.imlist.append(filename)
        
        self.transform = transform

  def __getitem__(self, index):

        impath = self.imlist[index]

        #собираем полное имя картинки
        full_imname = os.path.join(self.root, impath)

        if not os.path.exists(full_imname): #если нет такой, ругаемся
            print('No file ', full_imname)
            pass

        #Сразу используем PIL тк torchvision.transforms работает с PIL Image (https://pytorch.org/docs/stable/torchvision/transforms.html)
        img = Image.open(full_imname).convert('RGB')

        #применяем к изображению выбранное преобразование (аугментацию)
        img = self.transform(img)

        #на выход отдаём img - нужны для обучения и валидации
        return img, impath

  #метод возвращает длину датасета - просто как длину подаваемого dataframe
  def __len__(self):
        return len(self.imlist)

In [21]:
#преобразования для train и val
transform_for_train_and_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#преобразования для test, для старта те же
transform_for_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#датафреймы
train, val = train_test_split(train_df, test_size=0.1, random_state=42)

#нам нужно будет обращаться по индексу, так что делаем reset
train.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)

batch_size=64

In [22]:
trainset = RoadSignDataset(root='./data/train', flist=train, transform=transform_for_train_and_val)
valset = RoadSignDataset(root='./data/train', flist=val, transform=transform_for_train_and_val)
testset = RoadSignTestDataset(root='./data/test', flist=None, transform=transform_for_train_and_val)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, pin_memory=True)

In [16]:
def run_epoch(epoch, is_train):
  """
  Training and evaluaton loop over samples
  Args:
      train_mode (bool): True for train mode
  """
  if is_train:
      net.train()
      loader = trainloader
      print("Training epoch: ", epoch + 1, "/", num_epochs)
  else:
      net.eval()
      loader = valloader
      print('Validation')

  running_loss = 0.0
  correct = 0.0
  total = 0.0

  for i, data in enumerate(tqdm(loader)):
      # берем батч, кладем на GPU
      images, labels, _= data
      images, labels = images.cuda(), labels.cuda()

      #мы всегда прогоняем батч через сеть и считаем loss
      outputs = net(images)
      loss = criterion(outputs, labels)

      #занулять градиенты, считать новые, обновлять веса - всё нужно только на обучении
      if is_train:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      #пополняем логи
      running_loss += loss.item()
      total += images.data.size(0)

      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == labels.data).sum()

  #после эпохи или валидации логируем
  print('Loss: {:.3f}, accuracy: {:.3f}'.format(running_loss / (i + 1), correct / total * 100.0))

# Базовый эксперимент - ResNet18, без аугментаций, Adam, BatchSize 64

In [31]:
experiment = 'ResNet18_AugNone_Adam_LR3e4_BS64'

In [27]:
net = models.resnet18(pretrained=True).to(DEVICE)
lr = 3e-4 
num_epochs = 10 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

In [30]:
%%time
#а теперь просто запускаем функцию с разными значениями is_train для каждой эпохи
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/10 [00:00<?, ?it/s]

Training epoch:  1 / 10


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.701, accuracy: 84.830
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

...

Loss: 0.045, accuracy: 98.939
----------------------
Training epoch:  10 / 10


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.028, accuracy: 99.292
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

Loss: 0.082, accuracy: 98.506
----------------------
Finished training! Enjoy your results!
CPU times: total: 10min 16s
Wall time: 3min 16s


In [33]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=device)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 2 - ResNet50, без аугментаций, Adam, BatchSize 64

In [34]:
experiment = 'ResNet50_AugNone_Adam_LR3e4_BS64'

In [35]:
net = models.resnet50(pretrained=True).to(DEVICE)
lr = 3e-4 
num_epochs = 10 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\lakeo/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:09<00:00, 11.1MB/s]


In [36]:
%%time
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/10 [00:00<?, ?it/s]

Training epoch:  1 / 10


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.639, accuracy: 86.430
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

...

Loss: 0.069, accuracy: 98.428
----------------------
Training epoch:  10 / 10


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.024, accuracy: 99.489
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

Loss: 0.061, accuracy: 98.506
----------------------
Finished training! Enjoy your results!
CPU times: total: 12min 59s
Wall time: 4min 7s


In [37]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 3 - ResNet50, без аугментаций, Adam, BatchSize 64, CosineLR

In [42]:
experiment = 'ResNet50_AugNo_Adam_CosineLR01_BS64'

In [46]:
net = models.resnet50(pretrained=True).to(DEVICE)

lr = 0.01
num_epochs = 20

optimizer = optim.Adam(net.parameters(), lr)
criterion = nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(optimizer, T_max=int(len(trainset)/batch_size + 1)*num_epochs)

In [47]:
#впишем CosineLR в процесс обучения
def run_epoch(epoch, is_train):
  """
  Training and evaluaton loop over samples
  Args:
      train_mode (bool): True for train mode
  """
  if is_train:
      net.train()
      loader = trainloader
      print("Training epoch: ", epoch + 1, "/", num_epochs)
  else:
      net.eval()
      loader = valloader
      print('Validation')

  running_loss = 0.0
  correct = 0.0
  total = 0.0

  for i, data in enumerate(tqdm(loader)):
      images, labels, _= data
      images, labels = images.cuda(), labels.cuda()
      outputs = net(images)
      loss = criterion(outputs, labels)

      if is_train:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()

      running_loss += loss.item()
      total += images.data.size(0)

      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == labels.data).sum()

  lr = scheduler.optimizer.param_groups[0]['lr']
  print('Loss: {:.3f}, accuracy: {:.3f}, lr: {}'.format(running_loss / (i + 1), correct / total * 100.0, lr))

In [48]:
%%time
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/20 [00:00<?, ?it/s]

Training epoch:  1 / 20


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 2.329, accuracy: 45.723, lr: 0.009938441702975682
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

...

Loss: 0.071, accuracy: 98.310, lr: 6.155829702431139e-05
----------------------
Training epoch:  20 / 20


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.000, accuracy: 100.000, lr: 0.0
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

Loss: 0.071, accuracy: 98.270, lr: 0.0
----------------------
Finished training! Enjoy your results!
CPU times: total: 28min 18s
Wall time: 8min 10s


In [49]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 4 - обучаем на всех данных трейна (без валидации)

In [11]:
#преобразования для train и val
transform_for_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#преобразования для test, для старта те же
transform_for_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#датафреймы
train = train_df

#нам нужно будет обращаться по индексу, так что делаем reset
train.reset_index(inplace=True, drop=True)

batch_size=64

In [17]:
trainset = RoadSignDataset(root='./data/train', flist=None, transform=transform_for_train)
valset = RoadSignDataset(root='./data/train', flist=None, transform=transform_for_train)
testset = RoadSignTestDataset(root='./data/test', flist=None, transform=transform_for_train)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
valloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, pin_memory=True)

In [18]:
experiment = 'ResNet50_AugNone_Adam_LR3e4_BS64_FullTrain'

In [20]:
net = models.resnet50(pretrained=True).to(DEVICE)
lr = 3e-4 
num_epochs = 10 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

In [21]:
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/10 [00:00<?, ?it/s]

Training epoch:  1 / 10


  0%|          | 0/398 [00:00<?, ?it/s]

Loss: 0.583, accuracy: 87.280
Validation


  0%|          | 0/398 [00:00<?, ?it/s]

...

Loss: 0.016, accuracy: 99.501
----------------------
Training epoch:  10 / 10


  0%|          | 0/398 [00:00<?, ?it/s]

Loss: 0.028, accuracy: 99.316
Validation


  0%|          | 0/398 [00:00<?, ?it/s]

Loss: 0.010, accuracy: 99.725
----------------------
Finished training! Enjoy your results!


In [22]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 5. Добавим базовых аугментаций

In [28]:
experiment = 'ResNet50_AugBase_Adam_LR3e4_BS64'

In [36]:
#преобразования для train и val
transform_for_train = transforms.Compose([
        transforms.ColorJitter(brightness=(0.3, 1.0), contrast=(0.8, 1.2)),
        transforms.RandomRotation((-15, 15)), 
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#преобразования для test, для старта те же
transform_for_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#датафреймы
train, val = train_test_split(train_df, test_size=0.1, random_state=42)

#нам нужно будет обращаться по индексу, так что делаем reset
train.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)

batch_size=64

In [37]:
trainset = RoadSignDataset(root='./data/train', flist=train, transform=transform_for_train)
valset = RoadSignDataset(root='./data/train', flist=val, transform=transform_for_test)
testset = RoadSignTestDataset(root='./data/test', flist=None, transform=transform_for_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, pin_memory=True)

In [38]:
net = models.resnet50(pretrained=True).to(DEVICE)
lr = 3e-4 
num_epochs = 10 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

In [39]:
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/10 [00:00<?, ?it/s]

Training epoch:  1 / 10


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.770, accuracy: 83.144
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

...

Loss: 0.049, accuracy: 98.585
----------------------
Training epoch:  10 / 10


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.040, accuracy: 98.908
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

Loss: 0.082, accuracy: 98.074
----------------------
Finished training! Enjoy your results!


In [40]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 6. Аугментации с блюром

In [41]:
experiment = 'ResNet50_AugBaseBlur_Adam_LR3e4_BS64'

In [42]:
#преобразования для train и val
transform_for_train = transforms.Compose([
        transforms.ColorJitter(brightness=(0.3, 1.0), contrast=(0.8, 1.2)),
        transforms.RandomRotation((-15, 15)),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#преобразования для test, для старта те же
transform_for_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#датафреймы
train, val = train_test_split(train_df, test_size=0.1, random_state=42)

#нам нужно будет обращаться по индексу, так что делаем reset
train.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)

batch_size=64

In [43]:
trainset = RoadSignDataset(root='./data/train', flist=train, transform=transform_for_train)
valset = RoadSignDataset(root='./data/train', flist=val, transform=transform_for_test)
testset = RoadSignTestDataset(root='./data/test', flist=None, transform=transform_for_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, pin_memory=True)

In [44]:
net = models.resnet50(pretrained=True).to(DEVICE)
lr = 3e-4 
num_epochs = 20 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

In [45]:
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/20 [00:00<?, ?it/s]

Training epoch:  1 / 20


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.795, accuracy: 81.864
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

...

Loss: 0.042, accuracy: 98.821
----------------------
Training epoch:  20 / 20


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.030, accuracy: 99.161
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

Loss: 0.066, accuracy: 98.428
----------------------
Finished training! Enjoy your results!


In [46]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 7. Аугментации с блюром на ResNet152

In [52]:
experiment = 'ResNet152_AugBaseBlur_Adam_LR3e4_BS64'

In [48]:
#преобразования для train и val
transform_for_train = transforms.Compose([
        transforms.ColorJitter(brightness=(0.3, 1.0), contrast=(0.8, 1.2)),
        transforms.RandomRotation((-15, 15)),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#преобразования для test, для старта те же
transform_for_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

#датафреймы
train, val = train_test_split(train_df, test_size=0.1, random_state=42)

#нам нужно будет обращаться по индексу, так что делаем reset
train.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)

batch_size=64

In [50]:
trainset = RoadSignDataset(root='./data/train', flist=train, transform=transform_for_train)
valset = RoadSignDataset(root='./data/train', flist=val, transform=transform_for_test)
testset = RoadSignTestDataset(root='./data/test', flist=None, transform=transform_for_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, pin_memory=True)

In [51]:
net = models.resnet152(pretrained=True).to(DEVICE)
lr = 3e-4 
num_epochs = 20

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\lakeo/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|███████████████████████████████████████████████████████████████████████████████| 230M/230M [00:21<00:00, 11.5MB/s]


In [53]:
for epoch in tqdm(range(num_epochs)):
  #training
  run_epoch(epoch, is_train=True)

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False)

  print('----------------------')

print('Finished training! Enjoy your results!')

  0%|          | 0/20 [00:00<?, ?it/s]

Training epoch:  1 / 20


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.752, accuracy: 83.677
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

...

Loss: 0.053, accuracy: 98.664
----------------------
Training epoch:  20 / 20


  0%|          | 0/358 [00:00<?, ?it/s]

Loss: 0.027, accuracy: 99.244
Validation


  0%|          | 0/40 [00:00<?, ?it/s]

Loss: 0.053, accuracy: 98.270
----------------------
Finished training! Enjoy your results!


In [54]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]

# Эксперимент 8. Аугментации с блюром на ResNet152, CosineLR

In [56]:
experiment = 'ResNet152_AugBaseBlur_Adam_CosineLR01_BS64'

In [57]:
net = models.resnet152(pretrained=True).to(DEVICE)
lr = 0.01
num_epochs = 20

optimizer = optim.Adam(net.parameters(), lr)
criterion = nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(optimizer, T_max=int(len(trainset)/batch_size + 1)*num_epochs)

In [58]:
def run_epoch(epoch, is_train):
  """
  Training and evaluaton loop over samples
  Args:
      train_mode (bool): True for train mode
  """
  if is_train:
      net.train()
      loader = trainloader
      print("Training epoch: ", epoch + 1, "/", num_epochs)
  else:
      net.eval()
      loader = valloader
      print('Validation')
      
  running_loss = 0.0
  correct = 0.0
  total = 0.0

  for i, data in enumerate(loader):
      images, labels, _= data
      images, labels = images.cuda(), labels.cuda()
      outputs = net(images)
      loss = criterion(outputs, labels)
      
      if is_train:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()

      running_loss += loss.item()
      total += images.data.size(0)

      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == labels.data).sum()
  
  lr = scheduler.optimizer.param_groups[0]['lr']
  print('Loss: {:.3f}, accuracy: {:.3f}, lr: {}'.format(running_loss / (i + 1), correct / total * 100.0, lr))

In [59]:
for epoch in range(num_epochs):
  #training
  run_epoch(epoch, is_train=True) 

  #validation
  with torch.no_grad():
      run_epoch(epoch, is_train=False) 

  print('----------------------')

print('Finished training! Enjoy your results!')

Training epoch:  1 / 20
Loss: 3.337, accuracy: 19.486, lr: 0.009938441702975682
Validation
Loss: 3.639, accuracy: 34.277, lr: 0.009938441702975682
----------------------
Training epoch:  2 / 20
Loss: 1.886, accuracy: 47.846, lr: 0.009755282581475774
Validation
Loss: 2.299, accuracy: 61.635, lr: 0.009755282581475774
----------------------
Training epoch:  3 / 20
Loss: 1.035, accuracy: 70.290, lr: 0.009455032620941833
Validation
Loss: 2.681, accuracy: 71.069, lr: 0.009455032620941833
----------------------
Training epoch:  4 / 20
Loss: 0.670, accuracy: 80.540, lr: 0.009045084971874747
Validation
Loss: 0.671, accuracy: 81.014, lr: 0.009045084971874747
----------------------
Training epoch:  5 / 20
Loss: 0.479, accuracy: 85.914, lr: 0.00853553390593274
Validation
Loss: 1.454, accuracy: 85.731, lr: 0.00853553390593274
----------------------
Training epoch:  6 / 20
Loss: 0.372, accuracy: 89.016, lr: 0.007938926261462361
Validation
Loss: 0.396, accuracy: 88.325, lr: 0.007938926261462361
-----

In [60]:
predictions = {}
for batch_idx, (data, filenames) in enumerate(tqdm(testloader)):
    # Get data to Cuda/MPS
    data = data.to(device=DEVICE)
    scores = net(data)

    for filename, score in zip(filenames, scores):
        predictions[filename] = score.argmax().item()
preds_df = pd.DataFrame(predictions.items(), columns=['filename', 'class_number'])
preds_df.to_csv(f'./outputs/predictions_{experiment}.csv', index=False)

  0%|          | 0/118 [00:00<?, ?it/s]